<a href="https://colab.research.google.com/github/makhmudovamunira/Yuzdagi-niqobni-aniqlash/blob/main/Web_camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Colabga Driveni bog'laymiz**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Drive .json file manzilini chaqirib olamiz**

In [ ]:
!gdown --id 1_UnVbXhgr0i0N73WtG0f8I-TPaawR4H9

.json faylni to'g'ri manzilga joylashtiramiz

In [ ]:
import shutil
import os

#Kaggle API tokenini to'g'ri joyga joylashtirish
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

#Ruxsatlarni o'zgartirish
os.chmod('/root/.kaggle/kaggle.json', 600)

print('Kaggle API sozlandi')

Datasetni chaqiramiz

In [ ]:
!kaggle datasets download -d "andrewmvd/face-mask-detection"

Datasetni zip dan chiqarib olamiz

In [ ]:
import zipfile

dataset_path='/content/face-mask-detection.zip'
output_folder='/content/face-mask-dataset'

#Zip faylni chiqarish
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
  zip_ref.extractall(output_folder)

print(f"Dataset {output_folder} papkasiga chiqarildi")

In [ ]:
data_path='/content/face-mask-dataset'

print(os.listdir(data_path))

In [ ]:
print(len(os.listdir('/content/face-mask-dataset/annotations')))
print(len(os.listdir('/content/face-mask-dataset/images')))

In [ ]:
print(os.listdir('/content/face-mask-dataset/annotations'))

In [ ]:
import os
import xml.etree.ElementTree as ET
annotations_path='/content/face-mask-dataset/annotations'
class_counts={}

#Barcha XML fayllarni o'qish
for xml_file in os.listdir(annotations_path):
  if xml_file.endswith('.xml'):
    tree=ET.parse(os.path.join(annotations_path, xml_file))
    root = tree.getroot()

    for obj in root.findall('object'):
      class_name=obj.find('name').text
      class_counts[class_name] = class_counts.get(class_name, 0) + 1

#Natijalarni chiqarish
print('Annotatsiya natijalari: ')
for class_name, count in class_counts.items():
  print(f"{class_name} {count} ta")

In [ ]:
import random

# Ma'lumotlar yo‘llari
annotations_path = '/content/face-mask-dataset/annotations'
images_path = '/content/face-mask-dataset/images'

train_path='/content/face-mask-dataset-split/train'
test_path='/content/face-mask-dataset-split/test'

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)


for category in ['with_mask', 'without_mask']:
  os.makedirs(os.path.join(train_path, category), exist_ok=True)
  os.makedirs(os.path.join(test_path, category), exist_ok=True)

#Fayllarni saqlash uchun lug'atlar
data_by_class={'with_mask':[], 'without_mask':[]}

#Annotatsiyalarni o'qish

for xml_file in os.listdir(annotations_path):
  if xml_file.endswith('.xml'):
    tree=ET.parse(os.path.join(annotations_path, xml_file))
    root=tree.getroot()
    image_file=root.find('filename').text
    image_path=os.path.join(images_path, image_file)


    for obj in root.findall('object'):
      class_name=obj.find('name').text
      if class_name in data_by_class:
        data_by_class[class_name].append(image_path)

#Train va testlarga ajratish
for class_name, images in data_by_class.items():
  random.shuffle(images)
  split_idx=int(len(images)*0.8)

  train_files=images[:split_idx]
  test_files=images[split_idx:]

  #Fayllarni mos papkalarga ko'chiramiz
  for img in train_files:
    shutil.copy(img, os.path.join(train_path, class_name, os.path.basename(img)))

  for img in test_files:
    shutil.copy(img, os.path.join(test_path, class_name, os.path.basename(img)))

print('Dataset train va test papkalariga ajratildi')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Train va Test ma'lumotlari joylashgan papka
train_dir='/content/face-mask-dataset-split/train'
test_dir='/content/face-mask-dataset-split/test'

#Tasvirni oldindan qayta ishlash
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen=ImageDataGenerator(rescale=1./255)

#Tasvirlarni yuklash
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'
)

test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'
)


**CNN model yaratish**

In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5), #Overfitting oldini olish
    tf.keras.layers.Dense(1, activation='sigmoid') #2 ta class  uchun  sigmoidal chiqish
])

#Modelni kompilyatsiya qilish
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#Model tuzilmasini chiqarish
model.summary()

**Modelni train qilish**

In [ ]:
history=model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
test_loss, test_acc=model.evaluate(test_generator)
print(f"Test to'plamida aniqlik {test_acc*100:.2f}")

In [ ]:
model.save('face_mask_model.keras')
print('Model saqlandi')

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np

#Modelni yuklash
model=tf.keras.models.load_model('face_mask_model.keras')

#Classlar
classes=['With Mask', 'Without Mask']

#Kamera ochamiz
cap=cv2.VideoCapture(0)

while True:
  ret, frame=cap.read()
  if not ret:
    print("Kamerani ishga tushirib bo‘lmadi!")
    break

  #Tasvirni oldindan tayyorlash
  img=cv2.resize(frame, (128, 128))
  img=img/255.0
  img=np.expand_dims(img, axis=0)

  #Model bilan bashorat qilish
  prediction=model.predict(img)
  label=classes[np.argmax(prediction)]

  #Natijani ekranga chiqarish
  cv2.putText(frame, label, (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 2)
  cv2.imshow("Face Mask Detection", frame)

  #'q' tugmasi bosilsa dastur yopiladi
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import tensorflow as tf

# Modelni yuklash
model = tf.keras.models.load_model("face_mask_model.keras")

# Modelni qayta kompilyatsiya qilish
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Model qayta kompilyatsiya qilindi ✅")